In [ ]:
########## Step 1 Module and Parameters ##########
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset
from diffusers import DDPMScheduler, UNet2DModel 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sqlalchemy
from sklearn.preprocessing import StandardScaler
import os
import Engine.Model_S
import Engine.Model_M
import Engine.solver
from Engine.DATALOADER import build_dataloader_S, build_dataloader_M, UnNorm

class Args_Example:
    def __init__(self) -> None:
        self.device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f'Using device:{self.device}')
        self.save_dir = './results_CMAQ'
        self.checkpoint_model_dir = './chekcpoint/{modelfile}.pth' # model for sampling
        self.checkpoint_dir = './chekcpoint'
        self.input_dir = '{datapath}'  # .npy file with shape (timepoints,channels,width,length)
        self.multistep = 6             # steps of multistep verification in he training process
        self.width = 128
        self.length = 128
        self.seq_len = 1
        self.window = 2
        self.train_timestep = 24*361-73
        self.test_timestep = 24*4
        self.layers = 44
        self.channels = 66
        self.n_epoch = 50              # fewer for multistep verification in he training process
        self.gpu = 0
        self.ddimsteps = 10     
        self.batch_size = 32
        self.mode = None
        os.makedirs(self.save_dir, exist_ok=True)
        os.makedirs(self.checkpoint_dir, exist_ok=True)
args =  Args_Example() 

In [ ]:
########## Step 2 Train Diffusion Model (SingleStep) ##########
args.mode = 'train'
args.pmode = 'single'
model = Engine.Model_S.DDPM(args).to(args.device)
train_dataloader, scaler = build_dataloader_S(args)
Trainer = Engine.solver.Trainer(args, model, dataloader=train_dataloader)
print('Start Training......')
losses = Trainer.train_single()
print(len(losses))
print('Training Done!')
# Loss function
print(losses)
plt.plot(losses)
plt.ylim(0, 1)
plt.savefig(f"{args.save_dir}/loss_SingleStep.png", dpi=300, bbox_inches='tight')
print('Step 2 Done!')
losses = np.array(losses)
np.save(f'{args.save_dir}/loss_SingleStep.npy',losses)

In [ ]:
########## Step 2 Train Diffusion Model (MultiStep) ##########
args.mode = 'train'
args.pmode = 'multi'
args.R2 = np.load(f'{args.save_dir}/R2_pretrain.npy')[:(args.multistep)]
train_dataloader,aconc,weather = build_dataloader_M(args)
model = Engine.Model_M.DDPM(args,torch.tensor(aconc, dtype=torch.float32),torch.tensor(weather, dtype=torch.float32)).to(args.device)
Trainer = Engine.solver.Trainer(args, model, dataloader=train_dataloader)
print('Start Training......')
losses = Trainer.train_multi()
print(len(losses))
print('Training Done!')

# Loss function
plt.plot(losses)
plt.ylim(0, 1)
plt.savefig(f"{args.save_dir}/loss_MultiStep.png", dpi=300, bbox_inches='tight')
print('Step 2 Done!')
losses = np.array(losses)
np.save(f'{args.save_dir}/loss_MultiStep.npy',losses)

In [ ]:
########## Step 3 Sampling (SingleStep) ########## 
args.mode = 'sampling'
args.pmode = 'single'
model = Engine.Model_S.DDPM(args).to(args.device)
test_dataloader, scaler = build_dataloader_S(args)
Trainer = Engine.solver.Trainer(args, model, dataloader=test_dataloader)
real, sample = Trainer.sampling_single()
sample, real = UnNorm(sample, real, scaler)
np.savez(f'{args.save_dir}/prediction_ddpm_ss.npz', sample=sample, real=real)
print('Step 3 Done!')

In [ ]:
########## Step 3 Sampling (MuliStep) ########## 
args.mode = 'sampling'
args.pmode = 'multi'
model = Engine.Model_M.DDPM(args).to(args.device)
first,real,data,scaler = build_dataloader_M(args)
real = np.array(real).astype(np.float32)
sample = model.Reverse(data,first)
sample, real = UnNorm(sample, real, scaler)
np.savez(f'{args.save_dir}/prediction_ddpm_cc.npz', sample=sample, real=real)
print('Step 3 Done!')